## Borrower Reliability Study

The customer is the credit department of the bank. It is necessary to find out whether the marital status and the number of children of the client affect the fact of repaying the loan on time. Input data from the bank - statistics on the solvency of customers.

The results of the study will be taken into account when building a **credit scoring** model - a special system that evaluates the ability of a potential borrower to return a loan to the bank.

In [ ]:
import pandas as pd
from pymystem3 import Mystem

In [25]:
data = pd.read_csv('/Users/alexey_zalesov/Desktop/ya_prakrikum/ds/datasets/data.csv')
data.info()
print()
print(data.head(10))

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   children          21525 non-null  int64  
 1   days_employed     19351 non-null  float64
 2   dob_years         21525 non-null  int64  
 3   education         21525 non-null  object 
 4   education_id      21525 non-null  int64  
 5   family_status     21525 non-null  object 
 6   family_status_id  21525 non-null  int64  
 7   gender            21525 non-null  object 
 8   income_type       21525 non-null  object 
 9   debt              21525 non-null  int64  
 10  total_income      19351 non-null  float64
 11  purpose           21525 non-null  object 
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB

   children  days_employed  dob_years education  education_id  \
0         1   -8437.673028         42    высшее             0   
1         1   -4024.803754         36   

### Summary
Values of the same meaning in the 'purpose' column differ in spelling ("to play a wedding" and "to hold a wedding"); there are identical values that differ in case. In total, there are data on 21525 clients, and there are 19351 non-zero lines on employment and total income - there are gaps, respectively. There are negative values in the "days_employed" column.

## Data preprocessing

### Missing data

In [26]:
print(data.isnull().sum())
#вместо среднего арифметического меняем на медиану 
mean_days_empoyed = data['days_employed'].median()
mean_total_income = data['total_income'].median()
data['days_employed'] = data['days_employed'].fillna(mean_days_empoyed)
data['total_income'] = data['total_income'].fillna(mean_total_income)
print()
data.info() #проверка, что пропусков больше нет

children               0
days_employed       2174
dob_years              0
education              0
education_id           0
family_status          0
family_status_id       0
gender                 0
income_type            0
debt                   0
total_income        2174
purpose                0
dtype: int64

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   children          21525 non-null  int64  
 1   days_employed     21525 non-null  float64
 2   dob_years         21525 non-null  int64  
 3   education         21525 non-null  object 
 4   education_id      21525 non-null  int64  
 5   family_status     21525 non-null  object 
 6   family_status_id  21525 non-null  int64  
 7   gender            21525 non-null  object 
 8   income_type       21525 non-null  object 
 9   debt              21525 non-null  int64  
 10  total_in

### Summary
There are no gaps in the 'children', 'dob_years', 'education_id', 'income_type', 'debt' columns.
There is a strange 'XNA' value in the 'gender' column - possibly an upload error.
There are gaps in 'days_employed' and 'total_income'. There are 2174 of them - too many to simply remove. Since the number of gaps in both columns is the same, most likely the gaps are related: no information about the length of service - no income. The gaps have been replaced by the arithmetic mean of the respective columns.

### Changing data type

In [27]:
data['total_income'].astype('int')
data['days_employed'].astype('int')

0         -8437
1         -4024
2         -5623
3         -4124
4        340266
          ...  
21520     -4529
21521    343937
21522     -2113
21523     -3112
21524     -1984
Name: days_employed, Length: 21525, dtype: int64

### Summary
The values of both columns 'total_income' and 'days_employed' have been changed to integers. There are negative values in the 'days_employed' column, but since we do not need this column to answer the question, it was left unchanged. Since it is necessary to replace the real data type with an integer one, and the error resulting from changing the type is small, the astype() method is used.

### Duplicate data

In [28]:
data['education'].str.lower()
data['gender'].str.lower()
data['income_type'].str.lower()
data['purpose'].str.lower()
data.duplicated().sum() #проверяем количество дупликатов
data.drop_duplicates().reset_index(drop=True) #удаляем дупликаты 

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,-8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья
1,1,-4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля
2,0,-5623.422610,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья
3,3,-4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу
...,...,...,...,...,...,...,...,...,...,...,...,...
21466,1,-4529.316663,43,среднее,1,гражданский брак,1,F,компаньон,0,224791.862382,операции с жильем
21467,0,343937.404131,67,среднее,1,женат / замужем,0,F,пенсионер,0,155999.806512,сделка с автомобилем
21468,1,-2113.346888,38,среднее,1,гражданский брак,1,M,сотрудник,1,89672.561153,недвижимость
21469,3,-3112.481705,38,среднее,1,женат / замужем,0,M,сотрудник,1,244093.050500,на покупку своего автомобиля


### Summary
Since there are only 54 duplicates, and this is a very small proportion of all data, you can simply delete it using the drop_duplicates() method. Since the number of duplicates is very small, most likely, duplicates appeared in the process of uploading or entering information. 

### Lemmatisation

In [29]:
from pymystem3 import Mystem
m = Mystem()

lemmas_list = []
def lemmatize(data):
    lemmas = ''.join(m.lemmatize(data)).strip("\n")
    lemmas_list.append(lemmas)
    return lemmas
    
data['lemmatized_purpose'] =  data['purpose'].apply(lemmatize)
#lemmas_dict=set(lemmas_list)
#print(lemmas_dict) #смотрим леммы

def purpose_category(row):
    if 'свадьба' in row:
        return 'свадьба'
    if 'строительство' in row:
        return 'строительство'
    if 'недвижимость' in row:
        return 'недвижимость'
    if 'автомобиль' in row:
        return 'автомобиль'
    if 'ремонт' in row:
        return 'ремонт'
    if 'образование' in row:
        return 'образование'
    if 'жилье' in row:
        return 'недвижимость'
    pass


#создаем новый столбец с категориями целей получения кредита
data['categorized_purpose'] =  data['lemmatized_purpose'].apply(purpose_category) 
print(data.head())

print(data['categorized_purpose'].unique())





   children  days_employed  dob_years education  education_id  \
0         1   -8437.673028         42    высшее             0   
1         1   -4024.803754         36   среднее             1   
2         0   -5623.422610         33   Среднее             1   
3         3   -4124.747207         32   среднее             1   
4         0  340266.072047         53   среднее             1   

      family_status  family_status_id gender income_type  debt   total_income  \
0   женат / замужем                 0      F   сотрудник     0  253875.639453   
1   женат / замужем                 0      F   сотрудник     0  112080.014102   
2   женат / замужем                 0      M   сотрудник     0  145885.952297   
3   женат / замужем                 0      M   сотрудник     0  267628.550329   
4  гражданский брак                 1      F   пенсионер     0  158616.077870   

                      purpose          lemmatized_purpose categorized_purpose  
0               покупка жилья             

### Summary
Two new columns were created, lemmatization was carried out, and after that - categorization by goals (construction of real estate, education, repair (of a car or housing), holding a wedding, buying a car).

### Categorical Data

In [30]:
#создадим столбец kids_status - значение 1, если дети есть и 0, если их нет
def kids_status(kids):
    a=1
    b=0
    if kids >= 1:
        return a
    else:
        return b
data['kids_status'] = data['children'].apply(kids_status)
#print(data.head())
#создадим столбец marriage_status, 0 - не женат или не замужем, 1 - гражданский брак, 2 - женат или замежум, 3 - в разводе, 4 - вдовец или вдова. 
data['family_status'].value_counts()
def marriage_status(status):
    a = 0
    b = 1
    c = 2
    d = 3
    e = 4
    if status == 'женат / замужем':
        return 2
    if status == 'гражданский брак':
        return 1
    if status == 'Не женат / не замужем':
        return 0
    if status == 'в разводе':
        return 3
    else:
        return 4
data['marriage_status'] = data['family_status'].apply(marriage_status)
#создадим столбец income_status, категоризирующий данные по уровню дохода по 
data['total_income'].quantile([.25, .5, .75]) #находим границы квантилей
#0.25.   107798.172619

#0.50    156400.319836
#0.75    195543.620942

def income_type(income):
    if income<=107798.172619:
        return 'small'
    if 107798.172619<income<=156400.319836:
        return 'average'
    if 156400.319836<income<=195543.620942:
        return 'high'
    else:
        return 'top'

data['income_status'] = data['total_income'].apply(income_type)
print(data.head())

   children  days_employed  dob_years education  education_id  \
0         1   -8437.673028         42    высшее             0   
1         1   -4024.803754         36   среднее             1   
2         0   -5623.422610         33   Среднее             1   
3         3   -4124.747207         32   среднее             1   
4         0  340266.072047         53   среднее             1   

      family_status  family_status_id gender income_type  debt   total_income  \
0   женат / замужем                 0      F   сотрудник     0  253875.639453   
1   женат / замужем                 0      F   сотрудник     0  112080.014102   
2   женат / замужем                 0      M   сотрудник     0  145885.952297   
3   женат / замужем                 0      M   сотрудник     0  267628.550329   
4  гражданский брак                 1      F   пенсионер     0  158616.077870   

                      purpose          lemmatized_purpose categorized_purpose  \
0               покупка жилья            

### Summary
The categorization was carried out to answer the questions posed in the future. The "dictionaries" of these borrowers with and without children, according to the type of marital status, different income levels and purposes of obtaining a loan, are singled out.

###  Questions:

- Is there a relationship between having children and repaying a loan on time?

In [31]:
#для ответа на этот вопрос сравним соотношение должников с детьми и без детей к общему количеству должников
#общее количество должников
data[data['debt']==1].count() 
total_debtors = 1741
#общее количество должников, у которых есть дети
data[(data['debt']==1) & (data['kids_status']==1)].count()
debtors_with_kids = 677
#получается, должников без детей: 1741-677=1064
debtors_without_kids = 1064

debtors_part_with_kids = debtors_with_kids / total_debtors
debtors_part_without_kids = debtors_without_kids / total_debtors
#print(debtors_part_with_kids, 'Доля должников с детьми')
#print(debtors_part_without_kids, 'Доля должников без детей')


df_kids = data.pivot_table(index = 'debt', columns = 'kids_status', values = 'education', aggfunc = 'count'  )
df_kids = df_kids.transpose()
df_kids['total_clients'] = df_kids[1] + df_kids[0]
df_kids['debt_of_all_clients'] = df_kids[1] / df_kids['total_clients']
df_kids = df_kids.transpose()
print(df_kids)


kids_status                     0            1
debt                                          
0                    13132.000000  6652.000000
1                     1064.000000   677.000000
total_clients        14196.000000  7329.000000
debt_of_all_clients      0.074951     0.092373


### Summary
Since the proportion of debtors with children is much less than the proportion of debtors without children, it can be said that a borrower with children is more likely to repay a loan than a borrower without children.

- Is there a relationship between marital status and loan repayment on time?

In [32]:
#категории: 0 - не женат или не замужем, 
#1 - гражданский брак, 2 - женат или замужем, 3 - в разводе, 4 - вдовец или вдова. 
data[(data['debt']==1) & (data['marriage_status']==0)].count()
bachelors_debtors = 274
data[(data['debt']==1) & (data['marriage_status']==1)].count()
civil_marriage_debtors = 388
data[(data['debt']==1) & (data['marriage_status']==2)].count()
married_debtors = 931
data[(data['debt']==1) & (data['marriage_status']==3)].count()
divorced_debtors = 85
data[(data['debt']==1) & (data['marriage_status']==4)].count()
widow_debtors = 63

part_b_d = bachelors_debtors / total_debtors
part_c_d = civil_marriage_debtors / total_debtors
part_m_d = married_debtors / total_debtors
part_d_d = divorced_debtors / total_debtors
part_w_d = widow_debtors / total_debtors

print(part_b_d, 'доля должников, которые не женаты/не замужем')
print(part_c_d, 'доля должников, которые в гражданском браке')
print(part_m_d, 'доля должников, которые женаты')
print(part_d_d, 'доля должников в разводе')
print(part_w_d, 'доля должников, которые овдовели')

data_marriage_debt = data.pivot_table(index = 'debt', columns = 'marriage_status', values = 'education', aggfunc = 'count'  )
data_marriage_debt = data_marriage_debt.transpose()
data_marriage_debt['total_clients'] = data_marriage_debt[1] + data_marriage_debt[0]
data_marriage_debt['debt_of_all_clients'] = data_marriage_debt[1] / data_marriage_debt['total_clients']
data_marriage_debt = data_marriage_debt.transpose()
print(data_marriage_debt)


0.15738081562320505 доля должников, которые не женаты/не замужем
0.22286042504307868 доля должников, которые в гражданском браке
0.5347501435956347 доля должников, которые женаты
0.048822515795519814 доля должников в разводе
0.03618609994256174 доля должников, которые овдовели
marriage_status                0           1             2           3  \
debt                                                                     
0                    2539.000000  3789.00000  11449.000000  1110.00000   
1                     274.000000   388.00000    931.000000    85.00000   
total_clients        2813.000000  4177.00000  12380.000000  1195.00000   
debt_of_all_clients     0.097405     0.09289      0.075202     0.07113   

marriage_status               4  
debt                             
0                    897.000000  
1                     63.000000  
total_clients        960.000000  
debt_of_all_clients    0.065625  


### Summary
There is a relationship between marital status and loan repayment on time - the shares of debtors with different marital status differ significantly. Most likely, borrowers without a partner carry fewer risks and have fewer obligations than borrowers with a partner, and even more so borrowers who are married. Divorced people, most likely, are older than the first three groups, respectively, they are more likely to have a better education and carry fewer risks - they have more certainty. Widowed people are likely to be even older and carry even fewer risks. 

- Is there a relationship between income level and loan repayment on time?

In [33]:

data[(data['debt']==1) & (data['income_status']=='top')].count()
debtors_top_income = 386
data[(data['debt']==1) & (data['income_status']=='high')].count()
debtors_high_income = 456
data[(data['debt']==1) & (data['income_status']=='average')].count()
debtors_average_income = 472
data[(data['debt']==1) & (data['income_status']=='small')].count()
debtors_small_income = 427

print(debtors_top_income / total_debtors, 'доля должников с самым высоким уровнем дохода')
print(debtors_high_income / total_debtors, 'доля должников с  высоким уровнем дохода')
print(debtors_average_income / total_debtors, 'доля должников со средним уровнем дохода')
print(debtors_small_income / total_debtors, 'доля должников с маленьким уровнем дохода')

df_income_level = data.pivot_table(index = 'debt', columns = 'categorized_purpose', values = 'education', aggfunc = 'count')
df_income_level = df_income_level.transpose()
df_income_level['total_clients'] = df_income_level[1] + df_income_level[0]
df_income_level['debt_of_all_clients'] = df_income_level[1] / df_income_level['total_clients']
df_income_level = df_income_level.transpose()
print(df_income_level)

0.22171165996553704 доля должников с самым высоким уровнем дохода
0.2619184376794945 доля должников с  высоким уровнем дохода
0.27110855829982766 доля должников со средним уровнем дохода
0.24526134405514072 доля должников с маленьким уровнем дохода
categorized_purpose   автомобиль  недвижимость  образование     ремонт  \
debt                                                                     
0                    3912.000000   7744.000000  3652.000000  577.00000   
1                     403.000000    603.000000   370.000000   35.00000   
total_clients        4315.000000   8347.000000  4022.000000  612.00000   
debt_of_all_clients     0.093395      0.072242     0.091994    0.05719   

categorized_purpose      свадьба  строительство  
debt                                             
0                    2162.000000    1737.000000  
1                     186.000000     144.000000  
total_clients        2348.000000    1881.000000  
debt_of_all_clients     0.079216       0.076555  


### Summary
Debt is more likely to be repaid by a person in the first or last income quintile. This may be because it is easiest for them to maintain their income level, while the "middle" is more difficult and more likely to "lose" their current income level and fail to meet their income obligation.

- How do different purposes of a loan affect its repayment on time?

In [34]:
#сравним доли должников с разными целями
data[(data['debt']==1) & (data['categorized_purpose']=='свадьба')].count()
marriage_payment = 186
data[(data['debt']==1) & (data['categorized_purpose']=='автомобиль')].count()
buy_car = 403
data[(data['debt']==1) & (data['categorized_purpose']=='ремонт')].count()
repair_payment = 35
data[(data['debt']==1) & (data['categorized_purpose']=='образование')].count()
education_payment = 370
data[(data['debt']==1) & (data['categorized_purpose']=='строительство')].count()
building_estate = 144
data[(data['debt']==1) & (data['categorized_purpose']=='недвижимость')].count()  
estate_purchase = 603

print(estate_purchase/total_debtors, 'доля должников с целью покупки недвижимости')
print(buy_car/total_debtors, 'доля должников с целью покупки личного автомобиля')
print(building_estate/total_debtors, 'доля должников с целью строительства недвижимости')
print(repair_payment/total_debtors, 'доля должников с целью ремонта')
print(marriage_payment/total_debtors, 'доля должников с целью проведения свадьбы')
print(education_payment/total_debtors, 'доля должников с целью улучшения образования')

df_goals = data.pivot_table(index = 'debt', columns = 'categorized_purpose', values = 'education', aggfunc = 'count')
df_goals = df_goals.transpose()
df_goals['total_clients'] = df_goals[1] + df_goals[0]
df_goals['debt_of_all_clients'] = df_goals[1] / df_goals['total_clients']
df_goals = df_goals.transpose()
print(df_goals)

0.3463526708788053 доля должников с целью покупки недвижимости
0.231476163124641 доля должников с целью покупки личного автомобиля
0.08271108558299828 доля должников с целью строительства недвижимости
0.020103388856978748 доля должников с целью ремонта
0.10683515221137277 доля должников с целью проведения свадьбы
0.2125215393452039 доля должников с целью улучшения образования
categorized_purpose   автомобиль  недвижимость  образование     ремонт  \
debt                                                                     
0                    3912.000000   7744.000000  3652.000000  577.00000   
1                     403.000000    603.000000   370.000000   35.00000   
total_clients        4315.000000   8347.000000  4022.000000  612.00000   
debt_of_all_clients     0.093395      0.072242     0.091994    0.05719   

categorized_purpose      свадьба  строительство  
debt                                             
0                    2162.000000    1737.000000  
1                     186.

### Conclusion
The most non-refundable purposes of obtaining a loan: buying real estate, buying a car, improving education. Perhaps this is due to the fact that these are rather large loans, which are harder for customers to repay. Repairs, construction and weddings are not returned much less often, most likely, these are smaller expenses and it is easier for borrowers to repay them.

###   Summary

The most dangerous signs that indicate a high probability of non-repayment of the loan by the borrower are highlighted. Thanks to the analysis of the data provided, it is possible to make changes to the terms of the loan and reduce the number of non-performing loans.